# Face Detection Code

- This code was created using Mediapipe.

It's a code that uses Python and webcam to recognize faces in real time and put decorations on ears and nose.

The source is Python Api from Mediapipe. :)

## Library 

In [1]:
import cv2 
import mediapipe as mp

## Use Mediapipe Face Detection Api  
> Link : https://google.github.io/mediapipe/solutions/face_detection

In [ ]:
import cv2
import mediapipe as mp
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For static images:
IMAGE_FILES = []
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw face detections of each face.
    if not results.detections:
      continue
    annotated_image = image.copy()
    for detection in results.detections:
      print('Nose tip:')
      print(mp_face_detection.get_key_point(
          detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
      mp_drawing.draw_detection(annotated_image, detection)
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image)

    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.detections:
      for detection in results.detections:
        mp_drawing.draw_detection(image, detection)
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

## First, Face Detection Code
> Video : face_video.mp4(from pixels site)

In [ ]:
# Mediapipe, FaceDetection Code 

import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
# 얼굴 검출을 위한 face_detection 모듈을 사용

mp_drawing = mp.solutions.drawing_utils
# 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용


cap = cv2.VideoCapture('face_video.mp4')
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7) as face_detection:
    # model_selection = 0 <- 카메라에서 2미터 이내, 근거리
    # model_selection = 1 <- 카메라에서 5미터 정도에서 적합
    
    # min_detection_cofidence -> 0.7 일 경우, 70% 이상으로 인식
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            
            # 6개의 특징 추출 : 오른쪽 눈, 왼쪽 눈, 코 끝 부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기 
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈 
                nose_tip = keypoints[2] # 코 끝 
                
                h, w, _ = image.shape # height, width, channel
                # 이미지로부터 세로, 가로 크기 가져옴 
                
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                # 이미지 내에서 실제 좌표를 기입 (x, y)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                # 이미지 내에서 실제 좌표를 기입 (x, y)
                
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                # 코 끝 좌표를 기입 (x, y)
                
                # 양 눈에 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                # image, 오른쪽 눈에, 50크기, 파란색, 두께는 10, 부드러운 선
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
                # image, 왼쪽 눈에, 50크기, 초록색, 두께는 10, 부드러운 선
                
                cv2.circle(image, nose_tip, 30, (0, 255, 255), 10, cv2.LINE_AA)

                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

## My Detection Code (WebCam Code)

In [4]:
# Mediapipe, FaceDetection Code 

import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): 
    # 대상 이미지(3채널), x, y 좌표, width, height, 덮어 씌울 이미지(4채널)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0~1 사이로 변환 (1 : 불투명, 0 : 투명)
    
    try : 
        for c in range(0, 3) : # channel BGR (0,1,2 만큼)
            image[y-h : y+h, x-w : x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h : y+h, x-w : x+w, c] * (1 - mask_image))
    except : 
        pass 

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection
# 얼굴 검출을 위한 face_detection 모듈을 사용

mp_drawing = mp.solutions.drawing_utils
# 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 불러오기
cap = cv2.VideoCapture(0)

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 * 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 * 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 * 100 

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7) as face_detection:
    # model_selection = 0 <- 카메라에서 2미터 이내, 근거리
    # model_selection = 1 <- 카메라에서 5미터 정도에서 적합
    
    # min_detection_cofidence -> 0.7 일 경우, 70% 이상으로 인식
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
            
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            
            # 6개의 특징 추출 : 오른쪽 눈, 왼쪽 눈, 코 끝 부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                
                # 아래의 코드를 사용하게 될 경우, 얼굴을 인식하여 범위를 그려줍니다.
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기 
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈 
                nose_tip = keypoints[2] # 코 끝 
                
                h, w, _ = image.shape # height, width, channel
                # 이미지로부터 세로, 가로 크기 가져옴 
                
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                # 이미지 내에서 실제 좌표를 기입 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                # 이미지 내에서 실제 좌표를 기입 (x, y)
                
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                # 코 끝 좌표를 기입 (x, y)
                
#                 아래의 코드는 video.mp4으로 공부한 코드
#                 image[right_eye[1] - 50 : right_eye[1] + 50, right_eye[0] - 50 : right_eye[0] + 50] = image_right_eye 
#                 image[left_eye[1] - 50 : left_eye[1] + 50, left_eye[0] - 50 : left_eye[0] + 50] = image_left_eye 
#                 image[nose_tip[1] - 75 : nose_tip[1] + 75, nose_tip[0] - 125 : nose_tip[0] + 125] = image_nose 
                
                # overlay(image, x, y, w, h, overlay_image)
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 125, 75, image_nose)
                
        
        
        # Flip the image horizontally for a selfie-view display.
        image = cv2.resize(image, None, fx=0.7, fy=0.7)
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image,1))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1